In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
train_df = pd.read_csv(Path('Resources/2019loans.csv'))
test_df = pd.read_csv(Path('Resources/2020Q1loans.csv'))
train_df.head()

,Unnamed: 0,index,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,57107,57107,13375.0,0.1797,483.34,MORTGAGE,223000.0,Not Verified,low_risk,n,...,100.0,50.0,0.0,0.0,577150.0,122018.0,32000.0,170200.0,N,N
1,141451,141451,21000.0,0.1308,478.68,MORTGAGE,123000.0,Source Verified,low_risk,n,...,85.0,33.3,0.0,0.0,132750.0,27896.0,15900.0,35398.0,N,N
2,321143,321143,20000.0,0.1240,448.95,MORTGAGE,197000.0,Source Verified,low_risk,n,...,85.7,33.3,0.0,0.0,628160.0,114043.0,22600.0,90340.0,N,N
3,11778,11778,3000.0,0.1240,100.22,RENT,45000.0,Not Verified,low_risk,n,...,100.0,16.7,1.0,0.0,42006.0,20761.0,19900.0,15406.0,N,N
4,169382,169382,30000.0,0.1612,1056.49,MORTGAGE,133000.0,Source Verified,low_risk,n,...,100.0,66.7,0.0,0.0,283248.0,109056.0,79500.0,58778.0,N,N


In [3]:
# Convert categorical data to numeric and separate target feature for training data
from sklearn.preprocessing import LabelEncoder
X = train_df.drop('loan_status', axis = 1)
X_train = pd.get_dummies(X)
y = LabelEncoder().fit_transform(train_df['loan_status'])

In [4]:
# Convert categorical data to numeric and separate target feature for testing data
X2 = test_df.drop('loan_status',axis = 1)
X_test = pd.get_dummies(X2)
y2 = LabelEncoder().fit_transform(test_df['loan_status'])

In [5]:
# add missing dummy variables to testing set
for m in X_train.columns:
    if m not in X_test.columns:
        X_test[m] = 0

I predict that the Random Forest Classifier model will perform better than the Logistic Regression because there are so many features in these datasets (86 columns).

In [6]:
# Train the Logistic Regression model on the unscaled data and print the model score
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter = 800)
clf.fit(X_train,y)
print(f'the training score is: {clf.score(X_train,y)}')
print(f'the testing score is: {clf.score(X_test,y2)}')

the training score is: 0.6886699507389162
the testing score is: 0.5665674181199489


C:\Users\rache\anaconda3\envs\venv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [7]:
# Train a Random Forest Classifier model and print the model score
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier (n_estimators = 200, random_state = 1)
clf.fit(X_train,y)
print(f'the training score is: {clf.score(X_train,y)}')
print(f'the testing score is: {clf.score(X_test, y2)}')

the training score is: 1.0
the testing score is: 0.6210123351765207


As the result, Random Forest Classifier model performed better. But with 1.0 training score it's clearly overfitted.

In [8]:
# Scale the data
from sklearn.preprocessing import StandardScaler
scale = StandardScaler().fit(X_train)
X_train_scaled = scale.transform(X_train)
X_test_scaled = scale.transform(X_test)

In [9]:
# Train the Logistic Regression model on the scaled data and print the model score
clf = LogisticRegression(max_iter = 800)
clf.fit(X_train_scaled,y)
print(f'the training score is: {clf.score(X_train_scaled,y)}')
print(f'the testing score is: {clf.score(X_test_scaled,y2)}')

the training score is: 0.7126436781609196
the testing score is: 0.7203317737133135


In [10]:
# Train a Random Forest Classifier model on the scaled data and print the model score
clf = RandomForestClassifier (n_estimators = 200, random_state = 1)
clf.fit(X_train_scaled,y)
print(f'the training score is: {clf.score(X_train_scaled,y)}')
print(f'the testing score is: {clf.score(X_test_scaled, y2)}')

the training score is: 1.0
the testing score is: 0.6214376860910251


As I observed, scaled data really helped improve the results for Logistic Regression model. However, it did nothing to the Random Forest model. In this case, using the the Logistic Regression model with the scaled data would help us the most to predict the loan risk.